In [1]:
import os
from glob import glob
import numpy as np
import pandas as pd
import xarray as xr
import dask
import trend_utils as tu
from regridding import regrid
from utils import loca_gard_mapping, city_list, trend_metric_ids

In [2]:
################
#### Paths #####
################
# Update these for reproduction

project_data_path = "/storage/group/pches/default/users/dcl5300/conus_comparison_lafferty-etal-2024"
project_code_path = "/storage/home/dcl5300/work/current_projects/conus_comparison_lafferty-etal-2024"

In [3]:
############
### Dask ###
############
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    # account="pches",
    account="open",
    cores=1,
    memory="10GiB",
    walltime="00:10:00",
)

cluster.scale(jobs=20)  # ask for jobs

from dask.distributed import Client

client = Client(cluster)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.8.13:35635,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Fit trend

In [4]:
# Fit all: spatial
for metric_id in trend_metric_ids:
    tu.trend_fit_all(metric_id)

In [3]:
# Fit all: cities
for city in city_list:
    for metric_id in trend_metric_ids:
        tu.trend_fit_city(metric_id, city)

Error for chicago avg_tas ('INM-CM5-0', 'r4i1p1f1', 'ssp370', 'LOCA2'): data length is 71, expected 86
Error for chicago avg_tas ('IPSL-CM6A-LR', 'r3i1p1f1', 'ssp245', 'LOCA2'): data length is 72, expected 86
Error for chicago avg_tas ('IPSL-CM6A-LR', 'r4i1p1f1', 'ssp245', 'LOCA2'): data length is 71, expected 86
Error for chicago avg_tas ('IPSL-CM6A-LR', 'r5i1p1f1', 'ssp245', 'LOCA2'): data length is 64, expected 86
Error for chicago sum_hdd ('EC-Earth3', 'r130i1p1f1', 'ssp370', 'GARD-LENS'): zeros in data
Error for chicago sum_hdd ('INM-CM5-0', 'r4i1p1f1', 'ssp370', 'LOCA2'): zeros in data
Error for chicago sum_hdd ('IPSL-CM6A-LR', 'r3i1p1f1', 'ssp245', 'LOCA2'): zeros in data
Error for chicago sum_hdd ('IPSL-CM6A-LR', 'r4i1p1f1', 'ssp245', 'LOCA2'): zeros in data
Error for chicago sum_hdd ('IPSL-CM6A-LR', 'r5i1p1f1', 'ssp245', 'LOCA2'): zeros in data
Error for chicago sum_cdd ('EC-Earth3', 'r130i1p1f1', 'ssp370', 'GARD-LENS'): zeros in data
Error for chicago sum_cdd ('INM-CM5-0', 'r

## Regridding

In [4]:
%%time
# Regrid to LOCA using NN
method = "nearest"
target = "LOCA2"

for metric_id in metric_ids:
    # Save path
    store_path = f"{project_data_path}/trends/loca_grid/{metric_id}"
    # Get all files
    files = glob(f"{project_data_path}/trends/original_grid/{metric_id}/STAR-ESDM*") + \
            glob(f"{project_data_path}/trends/original_grid/{metric_id}/GARD-LENS*")

    # Loop through
    for file in files:
        # Check if done
        file_name = file.split('/')[-1]
        store_name = file_name.replace(".nc", f"_{method}.nc")
        if not os.path.exists(f"{store_path}/{store_name}"):
            try:
                # Regrid and store
                ds_tmp = xr.open_dataset(file)
                ds_out = regrid(ds_tmp, method=method, target=target)
                ds_out.to_netcdf(f"{store_path}/{store_name}")
            except Exception as e:
                print(file)

CPU times: user 37 s, sys: 9.46 s, total: 46.5 s
Wall time: 1min 43s


In [5]:
%%time
# Regrid to GARD-LENS using NN
method = "nearest"
target = "GARD-LENS"

for metric_id in metric_ids:
    # Save path
    store_path = f"{project_data_path}/trends/gard_grid/{metric_id}"
    # Get all files
    files = glob(f"{project_data_path}/trends/original_grid/{metric_id}/STAR-ESDM*") + \
            glob(f"{project_data_path}/trends/original_grid/{metric_id}/LOCA2*")

    # Loop through
    for file in files:
        # Check if done
        file_name = file.split('/')[-1]
        store_name = file_name.replace(".nc", f"_{method}.nc")
        if not os.path.exists(f"{store_path}/{store_name}"):
            try:
                # Regrid and store
                ds_tmp = xr.open_dataset(file)
                ds_out = regrid(ds_tmp, method=method, target=target)
                ds_out.to_netcdf(f"{store_path}/{store_name}")
            except Exception as e:
                print(file)

CPU times: user 38.2 s, sys: 9.07 s, total: 47.3 s
Wall time: 2min 14s
